In [1]:
import matplotlib
matplotlib.use('Qt5Agg')
from header import *
from mne.stats import spatio_temporal_cluster_1samp_test, spatio_temporal_cluster_test, permutation_cluster_1samp_test, permutation_cluster_test, summarize_clusters_stc
from scipy.stats.distributions import f,t
from tqdm import tqdm
import xarray as xr
#warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\Users\adorb\WinPython-64bit-3.6.3.0Qt5\python-3.6.3.amd64\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
t0 = time.perf_counter()
task = 'SMEG' #'MIMOSA'
states = ['RS','FA','OM']
subjects = get_subjlist(task)#, include_all=True)

no_blk2 = ['002', '004', '007', '016']
no_mri = ['019', '021']
reject = ['002', '004', '010', '011']
for sub in no_mri + reject:
    if sub in subjects:
        subjects.remove(sub)

subjects.sort()
experts = []
novices = []
experts_i = []
novices_i = []
for s,sub in enumerate(subjects):
    if expertise(sub) == 'N':
        novices.append(sub)
        novices_i.append(s)
    if expertise(sub) == 'E':
        experts.append(sub)
        experts_i.append(s)

In [14]:
PSD = xr.open_dataarray(op.join(Analysis_path, task, 'meg', 'Alpha', 'PSD.nc'))
PSD.load()
PSD = PSD.transpose('state', 'subject', 'freq', 'chan')
PSD_norm = PSD/PSD.sum(['freq', 'chan'])
print(PSD_norm)

<xarray.DataArray (state: 5, subject: 55, freq: 2049, chan: 275)>
array([[[[1.520515e-06, ..., 8.324049e-07],
         ...,
         [7.038912e-11, ..., 5.819002e-11]],

        ...,

        [[1.743167e-06, ..., 5.782697e-07],
         ...,
         [4.646666e-11, ..., 5.195834e-11]]],


       ...,


       [[[         nan, ...,          nan],
         ...,
         [         nan, ...,          nan]],

        ...,

        [[7.007803e-07, ..., 3.977057e-07],
         ...,
         [7.731861e-11, ..., 7.051495e-11]]]])
Coordinates:
  * subject  (subject) object '007' '012' '014' '016' ... '105' '108' '109'
  * state    (state) object 'RS1' 'FA1' 'FA2' 'OM1' 'OM2'
  * chan     (chan) object 'MLC11' 'MLC12' 'MLC13' ... 'MZO02' 'MZO03' 'MZP01'
  * freq     (freq) float64 0.0 0.1465 0.293 0.4395 ... 299.6 299.7 299.9 300.0


In [15]:
PSD_ave = np.empty((len(states), *PSD.shape[1:]))
for s,state in enumerate(states):
    PSD_ave[s] = PSD_norm.loc[fnmatch.filter(PSD_norm.state.values, state+'*')].mean('state').values
coords = {dim: PSD_norm.coords[dim].values for dim in PSD_norm.dims}
coords.update({'state': states})
PSD_ave = xr.DataArray(PSD_ave, dims=PSD_norm.dims, coords=coords)
print(PSD_ave)

<xarray.DataArray (state: 3, subject: 55, freq: 2049, chan: 275)>
array([[[[1.520515e-06, ..., 8.324049e-07],
         ...,
         [7.038912e-11, ..., 5.819002e-11]],

        ...,

        [[1.743167e-06, ..., 5.782697e-07],
         ...,
         [4.646666e-11, ..., 5.195834e-11]]],


       ...,


       [[[9.130033e-07, ..., 1.135848e-06],
         ...,
         [1.008259e-10, ..., 7.461664e-11]],

        ...,

        [[7.469942e-07, ..., 4.607345e-07],
         ...,
         [7.579208e-11, ..., 6.598015e-11]]]])
Coordinates:
  * state    (state) <U2 'RS' 'FA' 'OM'
  * subject  (subject) object '007' '012' '014' '016' ... '105' '108' '109'
  * freq     (freq) float64 0.0 0.1465 0.293 0.4395 ... 299.6 299.7 299.9 300.0
  * chan     (chan) object 'MLC11' 'MLC12' 'MLC13' ... 'MZO02' 'MZO03' 'MZP01'


In [16]:
def sensor_perm_test(X1, X2, stat_file, test_key, freqs, sensors, mode='a', p_threshold=0.01, connectivity=None, paired=False, fif_significance=0.05):
    """
    If paired, test X1-X2.
    A summary Evoked of the stats is saved if there is a significant cluster (p-value < fif_significance).
    (Time is replaced by freqs.)
    Saving can be forced by setting fif_significance to 1, or disabled by setting it to 0.
    Input: arrays of shape (subjects, freq, space)
    """
    os.makedirs(op.dirname(stat_file), exist_ok=True)
    evoked_file = op.splitext(stat_file)[0] + '_' + test_key + '-stat.fif'
    
    if not isinstance(X2, (np.ndarray, xr.DataArray, list)):
        X2 = np.zeros(X1.shape)
    
    if paired:
        X = X1 - X2
        t_threshold = -t.ppf(p_threshold / 2, X.shape[0] - 1)
        T_obs, clusters, cluster_pv, H0 = clu_all = spatio_temporal_cluster_1samp_test(X, connectivity=connectivity, threshold=t_threshold, n_jobs=4)
    else:
        f_threshold = f.ppf(1 - p_threshold / 2, X1.shape[0] - 1, X2.shape[0] - 1)
        T_obs, clusters, cluster_pv, H0 = clu_all = spatio_temporal_cluster_test([X1,X2], connectivity=connectivity, threshold=f_threshold, n_jobs=4)
    
    p_val = np.ones_like(T_obs)
    clu_inds = np.zeros_like(T_obs)
    info = mne.create_info(sensors, np.round(1 / (freqs[1] - freqs[0])), 'mag')
    
    for c,clu in enumerate(clusters):
        p_val[clu] = cluster_pv[c]
        clu_inds[clu] = c+1
        if np.where(cluster_pv[c] <= fif_significance)[0].size:
            data = np.full_like(T_obs, np.nan)
            data[clu] = T_obs[clu]
            mne.write_evokeds(evoked_file, mne.EvokedArray(data.T, info, freqs[0], 'cluster_{}'.format(c+1)))
    
    if np.where(p_val <= fif_significance)[0].size:
        mne.write_evokeds(evoked_file, mne.EvokedArray(np.where(p_val <= fif_significance, T_obs, np.nan).T, info, times[0], 'all_clusters'))
    
    stats = xr.DataArray(np.zeros((3, *T_obs.shape)), dims=['data', 'freq', 'sensor'], coords={'data':['T_stat', 'p_val', 'clu_inds'], 'freq':freqs, 'sensor':sensors})
    stats.loc['T_stat'] = T_obs
    stats.loc['p_val'] = p_val
    stats.loc['clu_inds'] = clu_inds
    
    stats.to_netcdf(path=stat_file, group=test_key, mode=mode if op.isfile(stat_file) else 'w')
    return clu_all

In [6]:
fmin = 2 #PSD_norm.freq.values[0]
fmax = 45 #PSD_norm.freq.values[-1]
stat_path = op.join(Analysis_path, task, 'meg', 'Stats', 'PSD')
os.makedirs(stat_path, exist_ok=True)
stat_file = op.join(stat_path, '{}-{}Hz.nc'.format(fmin, fmax))
paired_tests = {'RS_vs_FA':('RS', 'FA', subjects), 'RS_vs_OM':('RS', 'OM', subjects),
         'OM_vs_FA':('OM', 'FA', subjects), 'RS_vs_FA+E':('RS', 'FA', experts),
         'RS_vs_OM+E':('RS', 'OM', experts), 'OM_vs_FA+E':('OM', 'FA', experts),
         'RS_vs_FA+N':('RS', 'FA', novices), 'RS_vs_OM+N':('RS', 'OM', novices),
         'OM_vs_FA+N':('OM', 'FA', novices)}
clu = dict()

In [11]:
for key,val in paired_tests.items():
    clu[key] = sensor_perm_test(PSD_ave.loc[val[0],val[2],fmin:fmax].values, PSD_ave.loc[val[1],val[2],fmin:fmax].values, stat_file=stat_file, test_key=key, freqs=PSD_ave.loc[:,:,fmin:fmax].freq.values, sensors=PSD_ave.chan.values.tolist(), paired=True)

stat_fun(H1): min=-3.903630 max=4.331629
Running initial clustering
Found 48 clusters
Permuting 1023 times...


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    3.3s remaining:    3.3s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    5.5s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    5.5s finished


Computing cluster p-values
Done.
stat_fun(H1): min=-4.195056 max=3.370257
Running initial clustering
Found 27 clusters
Permuting 1023 times...


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    3.4s remaining:    3.4s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    4.7s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    4.7s finished


Computing cluster p-values
Done.
stat_fun(H1): min=-3.504096 max=3.761773
Running initial clustering
Found 47 clusters
Permuting 1023 times...


...

Computing cluster p-values
Done.
stat_fun(H1): min=-3.548634 max=4.546885
Running initial clustering
Found 22 clusters
Permuting 1023 times...


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    3.2s remaining:    3.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    4.7s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    4.7s finished


Computing cluster p-values
Done.
stat_fun(H1): min=-3.471529 max=4.742145
Running initial clustering
Found 47 clusters
Permuting 1023 times...


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    2.9s remaining:    2.9s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    4.9s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    4.9s finished


Computing cluster p-values
Done.
